In [1]:
!pip install lifelines scikit-learn joblib


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 9.6 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=92bcc8786b0f2923b9fdb12974b694f3eae883ed8f6dd50e4017b3f88a1a60f3
  Stored in directory: /root/.cache/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma


In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score, accuracy_score
from lifelines.utils import concordance_index


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

torch.manual_seed(42)
np.random.seed(42)


Using device: cpu


In [4]:
from google.colab import drive
drive.mount('/content/drive')

DUKE_IMG_EMB_DIR = (
    "/content/drive/MyDrive/permanent_data_folder/"
    "manifest-1654812109500/embeddings"
)

DUKE_CLINICAL_ARRAY = (
    "/content/drive/MyDrive/personalised survival treatment/"
    "embeddings/clinical_array.npy"
)

DUKE_MANIFEST = (
    "/content/drive/MyDrive/permanent_data_folder/"
    "manifest-1654812109500/manifest_matched.csv"
)


Mounted at /content/drive


In [5]:
import pandas as pd

DUKE_CLINICAL_RAW = (
    "/content/drive/MyDrive/personalised survival treatment/"
    "Clinical_and_Other_Features.xlsx"
)

clin_df = pd.read_excel(DUKE_CLINICAL_RAW)
print(clin_df.shape)
print(clin_df.columns)


(924, 98)
Index(['Patient Information', 'MRI Technical Information', 'Unnamed: 2',
       'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Demographics', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Tumor Characteristics', 'Unnamed: 24',
       'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36',
       'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40',
       'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44',
       'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'MRI Findings',
       'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 'SURGERY',
       'Unnamed: 54', 'Unnamed: 55', 'Radiation Therapy', 'Unnamed:

In [6]:
clin_df = pd.read_excel(
    DUKE_CLINICAL_RAW,
    header=1
)

print(clin_df.shape)
print(clin_df.columns.tolist())


(923, 98)
['Patient ID', 'Days to MRI (From the Date of Diagnosis)', 'Manufacturer', 'Manufacturer Model Name', 'Scan Options', 'Field Strength (Tesla)', 'Patient Position During MRI', 'Image Position of Patient', 'Contrast Agent', 'Contrast Bolus Volume (mL)', 'TR (Repetition Time)', 'TE (Echo Time)', 'Acquisition Matrix', 'Slice Thickness ', 'Rows', 'Columns', 'Reconstruction Diameter ', 'Flip Angle \n', 'FOV Computed (Field of View) in cm ', 'Date of Birth (Days)', 'Menopause (at diagnosis)', 'Race and Ethnicity', 'Metastatic at Presentation (Outside of Lymph Nodes)', 'ER', 'PR', 'HER2', 'Mol Subtype', 'Oncotype score', 'Staging(Tumor Size)# [T]', 'Staging(Nodes)#(Nx replaced by -1)[N]', 'Staging(Metastasis)#(Mx -replaced by -1)[M]', 'Tumor Grade', 'Unnamed: 32', 'Unnamed: 33', 'Nottingham grade', 'Histologic type', 'Tumor Location', 'Position', 'Bilateral Information', 'Unnamed: 39', 'Unnamed: 40', 'For Other Side If Bilateral', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed

In [7]:
for c in clin_df.columns:
    if any(x in c.lower() for x in ["patient", "er", "pr", "her2"]):
        print(c)


Patient ID
Manufacturer
Manufacturer Model Name
Patient Position During MRI
Image Position of Patient
Reconstruction Diameter 
Metastatic at Presentation (Outside of Lymph Nodes)
ER
PR
HER2
Bilateral Information
For Other Side If Bilateral
Contralateral Breast Involvement
Surgery
Days to Surgery (from the date of diagnosis)
Definitive Surgery Type
Neoadjuvant Radiation Therapy
Adjuvant Radiation Therapy
Pathologic Response to Neoadjuvant Therapy
Age at last contact in EMR f/u(days)(from the date of diagnosis) ,last time patient known to be alive, unless age of death is reported(in such case the age of death
Posterior acoustic shadowing
Neoadjuvant Chemotherapy
Adjuvant Chemotherapy
Neoadjuvant Endocrine Therapy Medications 
Adjuvant Endocrine Therapy Medications 
Number of Ovaries In Situ 

Therapeutic or Prophylactic Oophorectomy as part of Endocrine Therapy 
Neoadjuvant Anti-Her2 Neu Therapy
Adjuvant Anti-Her2 Neu Therapy 
Received Neoadjuvant Therapy or Not
Pathologic response to Ne

In [8]:
clin_df[["Patient ID", "ER", "PR", "HER2"]].head()


,Patient ID,ER,PR,HER2
0,NaN,"{0 = neg,\n1 = pos}","{0 = neg,\n1 = pos}","{0 = neg,\n1 = pos, 2 = borderline}"
1,Breast_MRI_001,0,0,1
2,Breast_MRI_002,0,0,0
3,Breast_MRI_003,1,1,0
4,Breast_MRI_004,1,1,0


In [9]:
def norm01(x):
    if pd.isna(x):
        return None
    try:
        x = int(x)
        if x in [0, 1]:
            return x
    except:
        pass
    return None

clin_df["ER_bin"] = clin_df["ER"].apply(norm01)
clin_df["PR_bin"] = clin_df["PR"].apply(norm01)
clin_df["HER2_bin"] = clin_df["HER2"].apply(norm01)


In [10]:
def derive_subtype(row):
    er, pr, her2 = row["ER_bin"], row["PR_bin"], row["HER2_bin"]

    # HER2+ dominates
    if her2 == 1:
        return "HER2+"

    # Hormone receptor positive
    if er == 1 or pr == 1:
        return "HR+/HER2-"

    # Triple negative
    if er == 0 and pr == 0 and her2 == 0:
        return "TripleNeg"

    return None


In [11]:
clin_df["subtype"] = clin_df.apply(derive_subtype, axis=1)

print(clin_df["subtype"].value_counts(dropna=False))


subtype
HR+/HER2-    595
TripleNeg    164
HER2+        163
None           1
Name: count, dtype: int64


In [12]:
subtype_df = clin_df[["Patient ID", "subtype"]].dropna()
subtype_df["Patient ID"] = subtype_df["Patient ID"].astype(str)

manifest["patient_id"] = manifest["patient_id"].astype(str)

manifest = manifest.merge(
    subtype_df,
    left_on="patient_id",
    right_on="Patient ID",
    how="inner"
)

print("Manifest shape after subtype merge:", manifest.shape)
print(manifest["subtype"].value_counts())


NameError: name 'manifest' is not defined

In [13]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
manifest["subtype_label"] = le.fit_transform(manifest["subtype"])

print(dict(zip(le.classes_, le.transform(le.classes_))))


NameError: name 'manifest' is not defined

In [ ]:
import pandas as pd
import numpy as np

clin_feat_df = pd.read_csv(
    "/content/drive/MyDrive/personalised survival treatment/clinical/clinical_processed.csv"
)

print(clin_feat_df.shape)
print(clin_feat_df.columns.tolist())


In [ ]:
manifest = pd.read_csv(DUKE_MANIFEST)

# REQUIRED columns in manifest:
# patient_id | img_file | time | event | subtype
print(manifest.head())
print(manifest.columns)


In [ ]:
# Load manifest (already done, but re-showing for clarity)
manifest = pd.read_csv(DUKE_MANIFEST)

# Load full clinical array (922 x 1730)
clinical_array = np.load(DUKE_CLINICAL_ARRAY)
print("Clinical array shape:", clinical_array.shape)

# Use clinical_row_index to align
clinical_indices = manifest["clinical_row_index"].values.astype(int) - 1

# Sanity check
assert clinical_indices.min() >= 0
assert clinical_indices.max() < clinical_array.shape[0]

# Aligned clinical features (169 x 1730)
clinical_aligned = clinical_array[clinical_indices].astype("float32")

print("Aligned clinical shape:", clinical_aligned.shape)


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(
    n_splits=1,
    test_size=0.2,
    random_state=42
)

for tr_idx, va_idx in sss.split(manifest, manifest["subtype_label"]):
    train_df = manifest.iloc[tr_idx].reset_index(drop=True)
    val_df   = manifest.iloc[va_idx].reset_index(drop=True)

train_clin = clinical_aligned[tr_idx]
val_clin   = clinical_aligned[va_idx]

print("Train:", train_df["subtype_label"].value_counts())
print("Val:", val_df["subtype_label"].value_counts())


In [ ]:
class DukeDataset(Dataset):
    def __init__(self, df, clin_array):
        self.df = df
        self.clin = clin_array

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img = np.load(
            row["image_feature_path_rep"]
        ).astype("float32")          # (512,)

        clin = self.clin[idx]

        return {
            "img": torch.tensor(img),
            "clin": torch.tensor(clin),
            "time": torch.tensor(row["time"], dtype=torch.float32),
            "event": torch.tensor(row["event"], dtype=torch.float32),
            "label": torch.tensor(row["subtype_label"], dtype=torch.long),
        }


In [ ]:
train_loader = DataLoader(
    DukeDataset(train_df, train_clin),
    batch_size=8,
    shuffle=True
)

val_loader = DataLoader(
    DukeDataset(val_df, val_clin),
    batch_size=8,
    shuffle=False
)


In [ ]:
def cox_ph_loss(risk, time, event):
    risk  = risk.view(-1)   # force (B,)
    time  = time.view(-1)
    event = event.view(-1)

    order = torch.argsort(time, descending=True)

    risk  = risk[order]
    event = event[order]

    log_cumsum = torch.logcumsumexp(risk, dim=0)
    loss = -((risk - log_cumsum) * event).sum()
    return loss / (event.sum() + 1e-8)


In [ ]:
# class DukeFusionModel(nn.Module):
#     def __init__(self, clin_dim, img_dim=512, num_classes=3):
#         super().__init__()

#         # Image branch
#         self.img_proj = nn.Sequential(
#             nn.Linear(img_dim, 256),
#             nn.ReLU()
#         )

#         # Clinical branch
#         self.clin_proj = nn.Sequential(
#             nn.Linear(clin_dim, 128),
#             nn.ReLU()
#         )

#         # 🔵 Survival uses FUSED features
#         self.surv_head = nn.Linear(256 + 128, 1)

#         # 🔵 Subtype uses IMAGE ONLY (leakage-safe)
#         self.cls_head = nn.Linear(256, num_classes)

#     def forward(self, clin, img):
#         img_feat  = self.img_proj(img)     # (B, 256)
#         clin_feat = self.clin_proj(clin)   # (B, 128)

#         fused = torch.cat([img_feat, clin_feat], dim=1)  # (B, 384)

#         risk = self.surv_head(fused)
#         logits = self.cls_head(img_feat)

#         return risk.squeeze(1), logits


import torch
import torch.nn as nn

class DukeTransformerSafe(nn.Module):
    def __init__(
        self,
        clin_dim,
        img_dim=512,
        d_model=256,
        num_heads=4,
        num_classes=3,
        dropout=0.1
    ):
        super().__init__()

        # ---- Projections ----
        self.img_proj = nn.Linear(img_dim, d_model)
        self.clin_proj = nn.Linear(clin_dim, d_model)

        # ---- Transformer for SURVIVAL (joint) ----
        surv_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=num_heads,
            dim_feedforward=2 * d_model,
            dropout=dropout,
            batch_first=True
        )
        self.surv_transformer = nn.TransformerEncoder(
            surv_layer, num_layers=1
        )

        # ---- Transformer for SUBTYPE (image only) ----
        subtype_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=num_heads,
            dim_feedforward=2 * d_model,
            dropout=dropout,
            batch_first=True
        )
        self.subtype_transformer = nn.TransformerEncoder(
            subtype_layer, num_layers=1
        )

        # ---- Heads ----
        self.surv_head = nn.Linear(d_model * 2, 1)
        self.cls_head = nn.Linear(d_model, num_classes)

    def forward(self, clin, img):
        img_tok = self.img_proj(img).unsqueeze(1)
        clin_tok = self.clin_proj(clin).unsqueeze(1)

        # ---- Survival path (joint attention) ----
        surv_tokens = torch.cat([img_tok, clin_tok], dim=1)
        surv_tokens = self.surv_transformer(surv_tokens)

        fused = torch.cat(
            [surv_tokens[:, 0, :], surv_tokens[:, 1, :]],
            dim=1
        )
        risk = self.surv_head(fused).squeeze(1)

        # ---- Subtype path (image-only attention) ----
        img_only = self.subtype_transformer(img_tok)
        logits = self.cls_head(img_only[:, 0, :])

        return risk, logits


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

classes = np.unique(train_df["subtype_label"])
weights = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=train_df["subtype_label"]
)

weights = torch.tensor(weights, dtype=torch.float32).to(device)

loss_cls = F.cross_entropy(logits, y, weight=weights)


In [ ]:
model = DukeTransformer(
    clin_dim=train_clin.shape[1],
    img_dim=512,
    num_classes=3
).to(device)


optimizer = torch.optim.Adam(
    model.parameters(),
    lr=3e-4,
    weight_decay=1e-4
)

alpha=0.5


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch

# ---- CLASS WEIGHTS (TRAIN SET ONLY) ----
classes = np.unique(train_df["subtype_label"].values)

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=train_df["subtype_label"].values
)

class_weights = torch.tensor(
    class_weights, dtype=torch.float32
).to(device)

print("Class weights:", class_weights)


In [ ]:
for epoch in range(20):
    model.train()
    total_loss = 0

    for b in train_loader:
        img   = b["img"].to(device)
        clin  = b["clin"].to(device)
        time  = b["time"].to(device)
        event = b["event"].to(device)
        y     = b["label"].to(device)

        risk, logits = model(clin, img)

        loss_surv = cox_ph_loss(risk, time, event)
        loss_cls = F.cross_entropy(logits, y, weight=class_weights)


        loss = loss_surv + alpha * loss_cls

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}: {total_loss/len(train_loader):.4f}")


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score
import torch.nn.functional as F

model.eval()

all_logits = []
all_labels = []

with torch.no_grad():
    for b in val_loader:
        img   = b["img"].to(device)
        clin  = b["clin"].to(device)

        _, logits = model(clin, img)

        all_logits.append(logits.cpu())
        all_labels.append(b["label"])

logits = torch.cat(all_logits)
labels = torch.cat(all_labels).numpy()

# ---- SHUFFLE LABELS ----
shuffled_labels = np.random.permutation(labels)

probs = F.softmax(logits, dim=1).numpy()

acc = accuracy_score(shuffled_labels, probs.argmax(axis=1))
auc = roc_auc_score(
    shuffled_labels,
    probs,
    multi_class="ovr",
    average="macro"
)

print("Accuracy after label shuffle:", acc)
print("ROC-AUC after label shuffle:", auc)


In [ ]:
model.eval()
all_risk, all_time, all_event = [], [], []
all_logits, all_labels = [], []

with torch.no_grad():
    for b in val_loader:
        img   = b["img"].to(device)
        clin  = b["clin"].to(device)

        risk, logits = model(clin, img)

        all_risk.extend(risk.cpu().numpy())
        all_time.extend(b["time"].numpy())
        all_event.extend(b["event"].numpy())

        all_logits.append(logits.cpu())
        all_labels.append(b["label"])


In [ ]:
from lifelines.utils import concordance_index
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np
import torch.nn.functional as F

model.eval()

all_risk   = []
all_time   = []
all_event  = []
all_logits = []
all_labels = []

with torch.no_grad():
    for b in val_loader:
        img   = b["img"].to(device)
        clin  = b["clin"].to(device)

        # IMPORTANT: unpack model outputs
        risk, logits = model(clin, img)

        # Survival
        all_risk.extend(risk.cpu().numpy())
        all_time.extend(b["time"].numpy())
        all_event.extend(b["event"].numpy())

        # Classification
        all_logits.append(logits.cpu())
        all_labels.append(b["label"])

# ------------------------
# SURVIVAL METRIC (C-INDEX)
# ------------------------
all_risk  = np.array(all_risk)
all_time  = np.array(all_time)
all_event = np.array(all_event)

c1 = concordance_index(all_time, all_risk, all_event)
c2 = concordance_index(all_time, -all_risk, all_event)

c_index = max(c1, c2)
print("Validation C-index:", c_index)

# ------------------------
# SUBTYPE METRICS (3-CLASS)
# ------------------------
logits = torch.cat(all_logits)
labels = torch.cat(all_labels).numpy()

probs = F.softmax(logits, dim=1).numpy()

# Macro-averaged One-vs-Rest ROC-AUC
roc_auc = roc_auc_score(
    labels,
    probs,
    multi_class="ovr",
    average="macro"
)

accuracy = accuracy_score(
    labels,
    probs.argmax(axis=1)
)

print("Subtype ROC-AUC (macro OVR):", roc_auc)
print("Subtype Accuracy:", accuracy)

# ------------------------
# SANITY CHECKS (OPTIONAL)
# ------------------------
print("Logits shape:", logits.shape)        # (N, 3)
print("Unique labels:", np.unique(labels))  # should be [0,1,2]
print("Prob sum (first row):", probs[0].sum())  # ≈ 1.0


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

X = clinical_aligned
y = manifest["subtype_label"].values

clf = LogisticRegression(max_iter=2000, multi_class="ovr")
scores = cross_val_score(clf, X, y, cv=5, scoring="accuracy")

print("CV accuracy (clinical only):", scores, scores.mean())


In [ ]:
y_shuffled = np.random.permutation(y)

scores_shuf = cross_val_score(
    clf,
    X,
    y_shuffled,
    cv=5,
    scoring="accuracy"
)

print("Shuffled accuracy:", scores_shuf, scores_shuf.mean())
